# Exploration des données de Sirene

## Importation des différents packages nécessaires.

In [ ]:
import pandas as pd
import dask.dataframe as dd
import random
import numpy as np
from datetime import datetime
from tqdm import tqdm
import nltk
import string
from nltk.corpus import stopwords
import fasttext
nltk.download('stopwords')

## Importation des données

In [ ]:
DBRaw = dd.read_parquet('../data/extraction_sirene_20220510.parquet', engine='pyarrow')

Temporaire : On restreint les données à 1% de la base initiales.

In [ ]:
DBRaw = DBRaw.sample(frac= 0.01, random_state=1234)

On transforme les valeurs manquantes en NaN.

In [ ]:
DBRaw = DBRaw.fillna(value=np.nan)

On garde seulement les variables potentiellement intéressantes : 
- ``APE_SICORE :`` Code APE (Activité Principale Exercée) retenu lors du traitement de codification (soit Sicore soit gestionnaire) ;
- ``NAT_SICORE :`` Nature de l'activité de l'entreprise ;
- ``SED_SICORE :`` Sédentarité de l'entreprise ;
- ``EVT_SICORE :`` Sédentarité de l'entreprise ;
- ``LIB_SICORE :`` Sédentarité de l'entreprise ;
- ``DATE :`` Sédentarité de l'entreprise ;
- ``AUTO :`` Type de liasse extrait de la base brute Sirène ;
- ``SURF :`` Surface en $m^2$ de l'établissement.


In [ ]:
Var2Keep = ["APE_SICORE","LIB_SICORE","AUTO","DATE","NAT_SICORE","SED_SICORE","EVT_SICORE","SURF"]
DB = DBRaw[Var2Keep]

On supprime les liasses où une valeur est manquante pour l'une de ces deux variables (6.77% de la base). Il s'agit principalement du code APE donc il n'est pas nécessaire de l'imputer.

In [ ]:
DB = DB.dropna(subset=['APE_SICORE'])  

On a finalement 10.8 millions de liasses.

In [ ]:
DB.shape[0].compute()

## I- Modèle 1

On estime un modèle FastText standard en utilisant seulement les libellés comme features.

### 1) Preprocessing 

In [ ]:
DB1 = DB[["APE_SICORE","LIB_SICORE"]]
DB1 = DB1.dropna(subset=['LIB_SICORE'])  

In [ ]:
stopwords_ = set(stopwords.words('french') + ['a'])
def CleanLib(lib):
    # On supprime toutes les ponctuations
    lib = lib.translate(str.maketrans(string.punctuation, ' ' * len(string.punctuation)))
    # On supprime tous les chiffres
    lib = lib.translate(str.maketrans(string.digits, ' ' * len(string.digits)))

    # On supprime les stopwords et on renvoie les mots en majuscule
    return " ".join([x.lower() for x in lib.split() if x.lower() not in stopwords_])

In [ ]:
DB1["LIB_CLEAN"] = DB1["LIB_SICORE"].apply(lambda x: CleanLib(x), meta=pd.Series(dtype='str', name='LIB_CLEAN'))
df = DB1.compute()

In [ ]:
df

### 2) Splitting

On mélange de manière aléatoires les index puis on les divise en 2 groupes selon un certain pourcentage (ici 80% et 20%)


In [ ]:
random.seed(123456)
Idx = random.sample(df.index.values.tolist(), df.shape[0])
Groups = np.split(Idx, [int(len(Idx)*0.8)])

In [ ]:
with open("../data/train_text.txt", 'w') as f:
    for idx in range(len(Groups[0])):
        aLine = "__label__{} {}".format(df.at[Groups[0][idx],"APE_SICORE"], df.at[Groups[0][idx],"LIB_CLEAN"])
        f.write("%s\n" % aLine)


### 3) Training

On définit plusieurs options pour le modèle.

In [ ]:
config_fasttext={"dim": 150,
"lr": 0.2,
"epoch": 80,
"wordNgrams": 3,
"minn": 3,
"maxn": 4,
"minCount": 3,
"bucket": 3000000,
"thread": 25,
"loss": 'ova',
"label_prefix": '__label__'}

In [ ]:
model1 = fasttext.train_supervised(input="../data/train_text.txt", **config_fasttext)

In [ ]:
TypeData = ["Train", "Test"]
dico_lib= { TypeData[i] : {Groups[i][idx] : {"TrueValue" : df.at[Groups[1][idx],"APE_SICORE"],
                                                "Libelle" : df.at[Groups[1][idx],"LIB_CLEAN"]} 
                                                for idx in range(len(Groups[1]))
                            } for i in range(len(TypeData))
}

In [ ]:
def get_pred(lib, mod):
    out = mod.predict(lib)
    pred = out[0][0].replace("__label__","")
    prob = out[1][0]
    return [pred, prob]

In [ ]:
df_test[['C', 'D']] = df_test['LIB_CLEAN'].apply(lambda x: get_pred(x, model1)).to_list()

In [ ]:
df_test

Accuracy of the training set

In [ ]:
df_train = df.loc[Groups[0], :]
df_train[['PREDICTION', 'PROBA']] = df_train['LIB_CLEAN'].apply(lambda x: get_pred(x, model1)).to_list()
df_train['GoodPREDICTION'] = df_train['APE_SICORE'] == df_train['PREDICTION']
sum(df_train['GoodPREDICTION'])/df_train.shape[0] * 100


Accuracy of the testing set

In [ ]:
df_test = df.loc[Groups[1], :]
df_test[['PREDICTION', 'PROBA']] = df_test['LIB_CLEAN'].apply(lambda x: get_pred(x, model1)).to_list()
df_test['GoodPREDICTION'] = df_test['APE_SICORE'] == df_test['PREDICTION']
sum(df_test['GoodPREDICTION'])/df_test.shape[0] * 100

In [ ]:
df_naf =  pd.read_csv(r"../data/naf_extended.csv",sep=",",encoding="utf-8",dtype=str)
df_naf["NIV5"] = df_naf["NIV5"].str.replace(".","")        
df_naf.set_index("NIV5", inplace=True)

In [ ]:
df_naf = df_naf[["NIV1","NIV2", "NIV3", "NIV4", "LIB_NIV5"]]

On regarde la precision le rappel et le F1 pour les différentes classes désaggrégées.

In [ ]:
df_naf = df_naf.join(df_train.groupby(['APE_SICORE']).size().rename("Size_TRAIN").to_frame())
df_naf = df_naf.join(df_test.groupby(['APE_SICORE']).size().rename("Size_TEST").to_frame())
df_naf = df_naf.join(df_test.groupby(['APE_SICORE']).mean('GoodPREDICTION').rename(columns={"GoodPREDICTION":"Recall_TEST"}))
df_naf = df_naf.join(df_test.groupby(['PREDICTION']).mean('GoodPREDICTION').rename(columns={"GoodPREDICTION":"Precision_TEST"}))
df_naf = df_naf.join(2 * 1/(1/df_naf["Precision_TEST"] + 1/df_naf["Recall_TEST"]).rename("F1_TEST").to_frame())

On fait la même chose au niveau moins aggrégé